In [0]:
import pandas as pd
from pandas import DataFrame, Series
import sqlite3 as db

In [0]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
col_list = ['age','workclass','fnlwgt','education','education-num','marital-status','occupation',
           'relationship','race','sex','capital-gain','capital-loss','hours-per-week','native-country','Label']
# Import the data from the url above into Pandas DataFrame
adult = pd.read_csv(url,sep=",",delimiter=",",names=col_list,skipinitialspace=True)

In [3]:
print(adult.columns)
# import regular expression library re.
# this is done in order to identify the column names with - in them and convert them to underscore "_" as hypen " - "
# is not a valid character in database
import re
adult.columns = [re.sub("[-]", "_", col) for col in adult.columns]

print(adult.columns)

Index(['age', 'workclass', 'fnlwgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'Label'],
      dtype='object')
Index(['age', 'workclass', 'fnlwgt', 'education', 'education_num',
       'marital_status', 'occupation', 'relationship', 'race', 'sex',
       'capital_gain', 'capital_loss', 'hours_per_week', 'native_country',
       'Label'],
      dtype='object')


In [4]:
# print first five rows of the dataframe head to chcek how the data loaded looks like
adult.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,Label
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


**Q1. Create an sqlalchemy engine using a sample from the data set**

In [0]:
# import corresponding sqlalchemy library
from sqlalchemy import *
from sqlalchemy import create_engine, ForeignKey
from sqlalchemy import Column, Date, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, backref

#create database engine
engine = create_engine('sqlite://', echo=True)

In [6]:
#connect to database and load  the dataframe adultdb into sqllite 
adult.to_sql('adultdb', con=engine)

2018-07-30 18:36:50,472 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-07-30 18:36:50,473 INFO sqlalchemy.engine.base.Engine ()
2018-07-30 18:36:50,478 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-07-30 18:36:50,481 INFO sqlalchemy.engine.base.Engine ()
2018-07-30 18:36:50,484 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("adultdb")
2018-07-30 18:36:50,486 INFO sqlalchemy.engine.base.Engine ()
2018-07-30 18:36:50,495 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE adultdb (
	"index" BIGINT, 
	age BIGINT, 
	workclass TEXT, 
	fnlwgt BIGINT, 
	education TEXT, 
	education_num BIGINT, 
	marital_status TEXT, 
	occupation TEXT, 
	relationship TEXT, 
	race TEXT, 
	sex TEXT, 
	capital_gain BIGINT, 
	capital_loss BIGINT, 
	hours_per_week BIGINT, 
	native_country TEXT, 
	"Label" TEXT
)


2018-07-30 18:36:50,498 INFO sqlalchemy.engine.base.Engine ()
2018-07-30 18:36:50,501 INFO sqlal

In [7]:
from sqlalchemy.orm import Session
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker


# create an Object to hold SQLAlchemy data types to map properties of Python classes into columns on a relation database table
Base = declarative_base(engine)
class Adultdb(Base):
    """
    eg. fields: id, title
    """
    __tablename__ = 'adultdb'
    __table_args__ = {'autoload': True}
    
    index = Column(Integer(), primary_key=True)
    age = Column(Integer())
    workclass = Column(String())

#Create SQLAlchemy Sessions

def loadSession():
    """"""
    metadata = Base.metadata
    Session = sessionmaker(bind=engine)
    session = Session()
    return session


2018-07-30 18:37:29,124 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("adultdb")
2018-07-30 18:37:29,126 INFO sqlalchemy.engine.base.Engine ()
2018-07-30 18:37:29,129 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'adultdb' AND type = 'table'
2018-07-30 18:37:29,130 INFO sqlalchemy.engine.base.Engine ()
2018-07-30 18:37:29,133 INFO sqlalchemy.engine.base.Engine PRAGMA foreign_key_list("adultdb")
2018-07-30 18:37:29,134 INFO sqlalchemy.engine.base.Engine ()
2018-07-30 18:37:29,136 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'adultdb' AND type = 'table'
2018-07-30 18:37:29,137 INFO sqlalchemy.engine.base.Engine ()
2018-07-30 18:37:29,140 INFO sqlalchemy.engine.base.Engine PRAGMA index_list("adultdb")
2018-07-30 18:37:29,142 INFO sqlalchemy.engine.base.Engine ()
2018-07-30 18:37:29,143 INFO sqlal

In [8]:
#Querying Data with SQLAlchemy ORM
if __name__ == "__main__":
    session = loadSession()
    rows = session.query(Adultdb).first()
    print("AGE","SEX   ","WORKCLAS ","COUNTRY ","OCCUPATION")
    print("*"*80)
    print(rows.age,rows.sex,rows.workclass,rows.native_country,rows.occupation)

2018-07-30 18:37:55,045 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-07-30 18:37:55,051 INFO sqlalchemy.engine.base.Engine SELECT adultdb."index" AS adultdb_index, adultdb.age AS adultdb_age, adultdb.workclass AS adultdb_workclass, adultdb.fnlwgt AS adultdb_fnlwgt, adultdb.education AS adultdb_education, adultdb.education_num AS adultdb_education_num, adultdb.marital_status AS adultdb_marital_status, adultdb.occupation AS adultdb_occupation, adultdb.relationship AS adultdb_relationship, adultdb.race AS adultdb_race, adultdb.sex AS adultdb_sex, adultdb.capital_gain AS adultdb_capital_gain, adultdb.capital_loss AS adultdb_capital_loss, adultdb.hours_per_week AS adultdb_hours_per_week, adultdb.native_country AS adultdb_native_country, adultdb."Label" AS "adultdb_Label" 
FROM adultdb
 LIMIT ? OFFSET ?
2018-07-30 18:37:55,053 INFO sqlalchemy.engine.base.Engine (1, 0)
AGE SEX    WORKCLAS  COUNTRY  OCCUPATION
************************************************************************

**Q2. Write two basic update queries **

Update the workclass and occupation to Study and Student respectively where fnlwgt = 77516

In [9]:
#Update Data with SQLAlchemy ORM
if __name__ == "__main__":
    session = loadSession()
    rows = session.query(Adultdb).filter_by(fnlwgt=77516).first()
    print(rows)
    rows.occupation ='Student'
    rows.workclass = 'Study'
    session.commit()

2018-07-30 18:40:28,094 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-07-30 18:40:28,097 INFO sqlalchemy.engine.base.Engine SELECT adultdb."index" AS adultdb_index, adultdb.age AS adultdb_age, adultdb.workclass AS adultdb_workclass, adultdb.fnlwgt AS adultdb_fnlwgt, adultdb.education AS adultdb_education, adultdb.education_num AS adultdb_education_num, adultdb.marital_status AS adultdb_marital_status, adultdb.occupation AS adultdb_occupation, adultdb.relationship AS adultdb_relationship, adultdb.race AS adultdb_race, adultdb.sex AS adultdb_sex, adultdb.capital_gain AS adultdb_capital_gain, adultdb.capital_loss AS adultdb_capital_loss, adultdb.hours_per_week AS adultdb_hours_per_week, adultdb.native_country AS adultdb_native_country, adultdb."Label" AS "adultdb_Label" 
FROM adultdb 
WHERE adultdb.fnlwgt = ?
 LIMIT ? OFFSET ?
2018-07-30 18:40:28,098 INFO sqlalchemy.engine.base.Engine (77516, 1, 0)
2018-07-30 18:40:28,101 INFO sqlalchemy.engine.base.Engine UPDATE adultdb SET wo

In [10]:
#verify update results
if __name__ == "__main__":
    session = loadSession()
    rows = session.query(Adultdb).filter_by(fnlwgt=77516).first()
    print("occupation : ",rows.occupation,"workclass :",rows.workclass,"fnlwgt : ",rows.fnlwgt)

2018-07-30 18:40:49,167 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-07-30 18:40:49,172 INFO sqlalchemy.engine.base.Engine SELECT adultdb."index" AS adultdb_index, adultdb.age AS adultdb_age, adultdb.workclass AS adultdb_workclass, adultdb.fnlwgt AS adultdb_fnlwgt, adultdb.education AS adultdb_education, adultdb.education_num AS adultdb_education_num, adultdb.marital_status AS adultdb_marital_status, adultdb.occupation AS adultdb_occupation, adultdb.relationship AS adultdb_relationship, adultdb.race AS adultdb_race, adultdb.sex AS adultdb_sex, adultdb.capital_gain AS adultdb_capital_gain, adultdb.capital_loss AS adultdb_capital_loss, adultdb.hours_per_week AS adultdb_hours_per_week, adultdb.native_country AS adultdb_native_country, adultdb."Label" AS "adultdb_Label" 
FROM adultdb 
WHERE adultdb.fnlwgt = ?
 LIMIT ? OFFSET ?
2018-07-30 18:40:49,174 INFO sqlalchemy.engine.base.Engine (77516, 1, 0)
occupation :  Student workclass : Study fnlwgt :  77516


Update occupation of every person who has a Masters as education and age =60 as Retired

In [11]:
#update query 2 
if __name__ == "__main__":
    session = loadSession()
    row = session.query(Adultdb).filter_by(age=60,education='Masters').all()
    for i in row:
        i.occupation = 'Retired'
    session.commit()

2018-07-30 18:41:29,950 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-07-30 18:41:29,954 INFO sqlalchemy.engine.base.Engine SELECT adultdb."index" AS adultdb_index, adultdb.age AS adultdb_age, adultdb.workclass AS adultdb_workclass, adultdb.fnlwgt AS adultdb_fnlwgt, adultdb.education AS adultdb_education, adultdb.education_num AS adultdb_education_num, adultdb.marital_status AS adultdb_marital_status, adultdb.occupation AS adultdb_occupation, adultdb.relationship AS adultdb_relationship, adultdb.race AS adultdb_race, adultdb.sex AS adultdb_sex, adultdb.capital_gain AS adultdb_capital_gain, adultdb.capital_loss AS adultdb_capital_loss, adultdb.hours_per_week AS adultdb_hours_per_week, adultdb.native_country AS adultdb_native_country, adultdb."Label" AS "adultdb_Label" 
FROM adultdb 
WHERE adultdb.age = ? AND adultdb.education = ?
2018-07-30 18:41:29,960 INFO sqlalchemy.engine.base.Engine (60, 'Masters')
2018-07-30 18:41:29,972 INFO sqlalchemy.engine.base.Engine UPDATE adultdb

In [12]:
#verify results after update

if __name__ == "__main__":
    session = loadSession()
    row = session.query(Adultdb).filter_by(age=60,education='Masters').all()
    for i in row:
        print(i.age,i.education,i.sex,i.occupation,i.workclass)

2018-07-30 18:41:50,230 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-07-30 18:41:50,232 INFO sqlalchemy.engine.base.Engine SELECT adultdb."index" AS adultdb_index, adultdb.age AS adultdb_age, adultdb.workclass AS adultdb_workclass, adultdb.fnlwgt AS adultdb_fnlwgt, adultdb.education AS adultdb_education, adultdb.education_num AS adultdb_education_num, adultdb.marital_status AS adultdb_marital_status, adultdb.occupation AS adultdb_occupation, adultdb.relationship AS adultdb_relationship, adultdb.race AS adultdb_race, adultdb.sex AS adultdb_sex, adultdb.capital_gain AS adultdb_capital_gain, adultdb.capital_loss AS adultdb_capital_loss, adultdb.hours_per_week AS adultdb_hours_per_week, adultdb.native_country AS adultdb_native_country, adultdb."Label" AS "adultdb_Label" 
FROM adultdb 
WHERE adultdb.age = ? AND adultdb.education = ?
2018-07-30 18:41:50,233 INFO sqlalchemy.engine.base.Engine (60, 'Masters')
60 Masters Male Retired ?
60 Masters Male Retired Self-emp-not-inc
60 Mas

**Q3. Write two delete queries**

In [13]:
#Check for all rows which have occupation as ? - Querying Data with SQLAlchemy ORM
if __name__ == "__main__":
    session = loadSession()
    rows = session.query(Adultdb).filter_by(occupation="?").all()
    print("Count of rows before delete operation : ",len(rows))

2018-07-30 18:42:37,723 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-07-30 18:42:37,726 INFO sqlalchemy.engine.base.Engine SELECT adultdb."index" AS adultdb_index, adultdb.age AS adultdb_age, adultdb.workclass AS adultdb_workclass, adultdb.fnlwgt AS adultdb_fnlwgt, adultdb.education AS adultdb_education, adultdb.education_num AS adultdb_education_num, adultdb.marital_status AS adultdb_marital_status, adultdb.occupation AS adultdb_occupation, adultdb.relationship AS adultdb_relationship, adultdb.race AS adultdb_race, adultdb.sex AS adultdb_sex, adultdb.capital_gain AS adultdb_capital_gain, adultdb.capital_loss AS adultdb_capital_loss, adultdb.hours_per_week AS adultdb_hours_per_week, adultdb.native_country AS adultdb_native_country, adultdb."Label" AS "adultdb_Label" 
FROM adultdb 
WHERE adultdb.occupation = ?
2018-07-30 18:42:37,727 INFO sqlalchemy.engine.base.Engine ('?',)
Count of rows before delete operation :  1841


In [14]:
#Delete rows which have occupation as "?" - Delete Data with SQLAlchemy ORM
if __name__ == "__main__":
    session = loadSession()
    session.query(Adultdb).filter_by(occupation="?").delete(synchronize_session='fetch')
    session.commit()
    rows = session.query(Adultdb).filter_by(occupation="?").all()
    print("Count of rows after delete operation : ",len(rows))

2018-07-30 18:43:06,451 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-07-30 18:43:06,453 INFO sqlalchemy.engine.base.Engine SELECT adultdb."index" AS adultdb_index 
FROM adultdb 
WHERE adultdb.occupation = ?
2018-07-30 18:43:06,454 INFO sqlalchemy.engine.base.Engine ('?',)
2018-07-30 18:43:06,465 INFO sqlalchemy.engine.base.Engine DELETE FROM adultdb WHERE adultdb.occupation = ?
2018-07-30 18:43:06,467 INFO sqlalchemy.engine.base.Engine ('?',)
2018-07-30 18:43:06,478 INFO sqlalchemy.engine.base.Engine COMMIT
2018-07-30 18:43:06,481 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-07-30 18:43:06,482 INFO sqlalchemy.engine.base.Engine SELECT adultdb."index" AS adultdb_index, adultdb.age AS adultdb_age, adultdb.workclass AS adultdb_workclass, adultdb.fnlwgt AS adultdb_fnlwgt, adultdb.education AS adultdb_education, adultdb.education_num AS adultdb_education_num, adultdb.marital_status AS adultdb_marital_status, adultdb.occupation AS adultdb_occupation, adultdb.relations

In [15]:
#Delete 2
#Check for all rows which have education as  Some-college- Querying Data with SQLAlchemy ORM
if __name__ == "__main__":
    session = loadSession()
    rows = session.query(Adultdb).filter_by(education="Some-college").all()
    print("count of rows :",len(rows))

2018-07-30 18:43:30,316 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-07-30 18:43:30,321 INFO sqlalchemy.engine.base.Engine SELECT adultdb."index" AS adultdb_index, adultdb.age AS adultdb_age, adultdb.workclass AS adultdb_workclass, adultdb.fnlwgt AS adultdb_fnlwgt, adultdb.education AS adultdb_education, adultdb.education_num AS adultdb_education_num, adultdb.marital_status AS adultdb_marital_status, adultdb.occupation AS adultdb_occupation, adultdb.relationship AS adultdb_relationship, adultdb.race AS adultdb_race, adultdb.sex AS adultdb_sex, adultdb.capital_gain AS adultdb_capital_gain, adultdb.capital_loss AS adultdb_capital_loss, adultdb.hours_per_week AS adultdb_hours_per_week, adultdb.native_country AS adultdb_native_country, adultdb."Label" AS "adultdb_Label" 
FROM adultdb 
WHERE adultdb.education = ?
2018-07-30 18:43:30,322 INFO sqlalchemy.engine.base.Engine ('Some-college',)
count of rows : 6775


In [16]:
#delete rows from table adultdb which have education as "Some-college" - Delete Data with SQLAlchemy ORM
#delete(synchronize_session='fetch')
#'fetch' - performs a select query before the delete to find objects that are matched by the delete query and 
#need to be removed from the session. Matched objects are removed from the session
if __name__ == "__main__":
    session = loadSession()
    session.query(Adultdb).filter_by(education="Some-college").delete(synchronize_session='fetch')
    session.commit()
    rows = session.query(Adultdb).filter_by(education="Some-college").all()
    print("Count of rows after Delete : ",len(rows))

2018-07-30 18:43:53,450 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-07-30 18:43:53,453 INFO sqlalchemy.engine.base.Engine SELECT adultdb."index" AS adultdb_index 
FROM adultdb 
WHERE adultdb.education = ?
2018-07-30 18:43:53,454 INFO sqlalchemy.engine.base.Engine ('Some-college',)
2018-07-30 18:43:53,468 INFO sqlalchemy.engine.base.Engine DELETE FROM adultdb WHERE adultdb.education = ?
2018-07-30 18:43:53,469 INFO sqlalchemy.engine.base.Engine ('Some-college',)
2018-07-30 18:43:53,486 INFO sqlalchemy.engine.base.Engine COMMIT
2018-07-30 18:43:53,489 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-07-30 18:43:53,491 INFO sqlalchemy.engine.base.Engine SELECT adultdb."index" AS adultdb_index, adultdb.age AS adultdb_age, adultdb.workclass AS adultdb_workclass, adultdb.fnlwgt AS adultdb_fnlwgt, adultdb.education AS adultdb_education, adultdb.education_num AS adultdb_education_num, adultdb.marital_status AS adultdb_marital_status, adultdb.occupation AS adultdb_occupatio


**Q4. Write two filter queries**

In [17]:
# Querying Data with SQLAlchemy ORM based on filter on workclass column having a value as "Private"
if __name__ == "__main__":
    session = loadSession()
    rows = session.query(Adultdb).filter_by(workclass='Private').all()
    for i in rows:
        print(i.age,i.sex,i.workclass,i.native_country)

2018-07-30 18:44:26,947 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-07-30 18:44:26,953 INFO sqlalchemy.engine.base.Engine SELECT adultdb."index" AS adultdb_index, adultdb.age AS adultdb_age, adultdb.workclass AS adultdb_workclass, adultdb.fnlwgt AS adultdb_fnlwgt, adultdb.education AS adultdb_education, adultdb.education_num AS adultdb_education_num, adultdb.marital_status AS adultdb_marital_status, adultdb.occupation AS adultdb_occupation, adultdb.relationship AS adultdb_relationship, adultdb.race AS adultdb_race, adultdb.sex AS adultdb_sex, adultdb.capital_gain AS adultdb_capital_gain, adultdb.capital_loss AS adultdb_capital_loss, adultdb.hours_per_week AS adultdb_hours_per_week, adultdb.native_country AS adultdb_native_country, adultdb."Label" AS "adultdb_Label" 
FROM adultdb 
WHERE adultdb.workclass = ?
2018-07-30 18:44:26,956 INFO sqlalchemy.engine.base.Engine ('Private',)
38 Male Private United-States
53 Male Private United-States
28 Female Private Cuba
37 Female Pri

38 Male Private United-States
64 Male Private United-States
46 Male Private United-States
39 Male Private United-States
48 Female Private United-States
46 Female Private United-States
42 Male Private United-States
24 Female Private United-States
30 Female Private Canada
23 Female Private United-States
32 Male Private United-States
55 Male Private United-States
18 Male Private United-States
40 Male Private United-States
27 Male Private United-States
55 Female Private United-States
36 Male Private United-States
26 Female Private United-States
18 Male Private United-States
38 Male Private United-States
50 Male Private United-States
57 Male Private United-States
20 Male Private United-States
60 Female Private Puerto-Rico
52 Female Private United-States
61 Male Private United-States
31 Male Private United-States
42 Male Private United-States
47 Male Private ?
27 Male Private United-States
25 Female Private United-States
51 Male Private United-States
28 Female Private ?
35 Male Private Unite

 Private United-States
52 Female Private China
39 Male Private United-States
33 Male Private United-States
46 Female Private Italy
35 Female Private United-States
33 Male Private United-States
48 Male Private United-States
32 Male Private United-States
35 Male Private United-States
19 Female Private United-States
21 Female Private ?
56 Female Private United-States
28 Female Private El-Salvador
45 Female Private United-States
32 Female Private United-States
44 Male Private United-States
55 Male Private United-States
33 Male Private United-States
36 Male Private United-States
25 Female Private United-States
28 Male Private United-States
39 Male Private United-States
53 Female Private Columbia
47 Female Private United-States
22 Male Private United-States
19 Male Private United-States
44 Male Private United-States
23 Male Private United-States
51 Male Private United-States
41 Male Private United-States
22 Male Private United-States
22 Female Private United-States
49 Male Private United-Sta

 Female Private United-States
50 Male Private United-States
57 Female Private United-States
25 Male Private United-States
20 Female Private United-States
46 Female Private United-States
54 Female Private Puerto-Rico
21 Male Private United-States
30 Male Private United-States
61 Female Private United-States
59 Female Private United-States
72 Male Private United-States
33 Male Private United-States
46 Female Private United-States
48 Female Private United-States
33 Male Private United-States
27 Female Private United-States
31 Male Private United-States
28 Male Private United-States
51 Male Private United-States
17 Female Private United-States
26 Male Private United-States
35 Male Private United-States
37 Female Private ?
42 Female Private United-States
35 Male Private United-States
45 Female Private United-States
64 Male Private United-States
39 Female Private United-States
34 Male Private ?
62 Male Private United-States
31 Male Private United-States
26 Male Private United-States
26 Male 

Private United-States
58 Male Private United-States
38 Female Private Philippines
18 Female Private United-States
40 Male Private United-States
35 Male Private ?
45 Male Private Mexico
23 Male Private United-States
28 Female Private United-States
51 Male Private United-States
44 Female Private United-States
24 Male Private United-States
24 Male Private United-States
52 Male Private Puerto-Rico
43 Male Private United-States
31 Male Private ?
36 Male Private United-States
43 Male Private United-States
31 Male Private United-States
26 Male Private United-States
49 Male Private United-States
34 Female Private United-States
29 Female Private United-States
50 Female Private United-States
66 Male Private Philippines
60 Male Private United-States
60 Female Private United-States
19 Male Private United-States
65 Male Private United-States
23 Female Private United-States
21 Male Private United-States
50 Male Private United-States
32 Female Private United-States
40 Male Private United-States
19 Ma

...

Private Vietnam
24 Male Private United-States
40 Male Private United-States
71 Male Private United-States
30 Female Private United-States
34 Male Private United-States
38 Female Private United-States
43 Male Private United-States
55 Female Private United-States
51 Male Private United-States
28 Male Private United-States
54 Male Private United-States
47 Male Private United-States
19 Female Private United-States
24 Female Private United-States
28 Male Private United-States
61 Male Private United-States
27 Female Private United-States
41 Female Private United-States
36 Male Private United-States
51 Female Private United-States
19 Female Private United-States
57 Female Private United-States
51 Female Private United-States
44 Male Private United-States
40 Male Private United-States
46 Female Private United-States
33 Male Private United-States
22 Female Private United-States
43 Female Private United-States
46 Male Private United-States
59 Male Private United-States
23 Male Private United-Sta

 Male Private United-States
31 Male Private United-States
45 Male Private United-States
34 Male Private Mexico
68 Female Private United-States
51 Male Private United-States
55 Male Private United-States
43 Male Private United-States
48 Male Private United-States
32 Male Private United-States
17 Male Private United-States
37 Female Private United-States
26 Female Private Dominican-Republic
17 Female Private United-States
34 Male Private United-States
46 Male Private United-States
33 Male Private ?
59 Male Private India
46 Male Private United-States
45 Male Private United-States
27 Female Private United-States
31 Male Private United-States
32 Female Private United-States
28 Male Private United-States
32 Male Private United-States
36 Male Private United-States
23 Male Private United-States
44 Male Private United-States
36 Female Private Puerto-Rico
45 Female Private South
49 Female Private United-States
37 Male Private United-States
26 Male Private United-States
34 Male Private United-Sta

 Private United-States
21 Female Private United-States
48 Female Private El-Salvador
51 Male Private United-States
43 Male Private United-States
38 Male Private ?
44 Male Private United-States
35 Female Private United-States
40 Male Private United-States
39 Female Private United-States
23 Female Private United-States
26 Male Private United-States
30 Male Private United-States
39 Male Private United-States
37 Female Private United-States
59 Male Private United-States
28 Male Private United-States
47 Male Private Cuba
29 Male Private United-States
40 Female Private United-States
48 Male Private United-States
50 Male Private United-States
23 Male Private United-States
35 Male Private United-States
39 Male Private United-States
52 Male Private Cuba
44 Male Private United-States
43 Male Private United-States
39 Male Private United-States
33 Male Private Mexico
31 Male Private United-States
19 Female Private Thailand
35 Male Private United-States
44 Male Private United-States
55 Male Private


41 Male Private United-States
23 Female Private United-States
17 Male Private United-States
32 Male Private United-States
42 Male Private United-States
68 Male Private United-States
43 Male Private United-States
61 Male Private United-States
28 Male Private United-States
47 Female Private United-States
39 Male Private United-States
27 Male Private United-States
25 Male Private Mexico
35 Female Private United-States
27 Male Private United-States
36 Male Private United-States
42 Male Private United-States
21 Female Private United-States
30 Male Private United-States
53 Female Private United-States
34 Male Private Germany
37 Male Private United-States
27 Female Private United-States
31 Male Private United-States
33 Male Private United-States
30 Female Private United-States
51 Male Private United-States
49 Male Private United-States
42 Male Private United-States
62 Female Private United-States
50 Male Private United-States
33 Male Private India
31 Male Private Mexico
56 Male Private Unite

 Male Private United-States
43 Female Private United-States
53 Male Private United-States
37 Male Private United-States
35 Male Private United-States
29 Male Private United-States
35 Female Private United-States
40 Female Private United-States
31 Male Private United-States
25 Female Private United-States
38 Male Private United-States
52 Male Private United-States
23 Male Private Mexico
46 Male Private United-States
26 Male Private United-States
47 Male Private United-States
65 Male Private United-States
38 Female Private United-States
68 Male Private United-States
26 Female Private Hong
42 Male Private United-States
49 Male Private United-States
42 Female Private Philippines
39 Male Private United-States
36 Female Private United-States
62 Male Private United-States
21 Male Private United-States
30 Male Private United-States
64 Female Private United-States
28 Female Private United-States
25 Male Private United-States
27 Male Private United-States
40 Male Private United-States
31 Female 

In [18]:
# Querying Data with SQLAlchemy ORM based on filter on marital_status column having a value as "Never-married"
count = 0
for rows in engine.execute("SELECT * FROM adultdb where marital_status == 'Never-married'").fetchall():
    print(rows.age,",",rows.marital_status,",",rows.occupation)
    count+=1
    
print("Total Number of rows fetched : ",count)

2018-07-30 18:46:43,237 INFO sqlalchemy.engine.base.Engine SELECT * FROM adultdb where marital_status == 'Never-married'
2018-07-30 18:46:43,239 INFO sqlalchemy.engine.base.Engine ()
39 , Never-married , Student
31 , Never-married , Prof-specialty
23 , Never-married , Adm-clerical
32 , Never-married , Sales
25 , Never-married , Farming-fishing
32 , Never-married , Machine-op-inspct
19 , Never-married , Craft-repair
23 , Never-married , Protective-serv
48 , Never-married , Machine-op-inspct
25 , Never-married , Handlers-cleaners
29 , Never-married , Prof-specialty
18 , Never-married , Other-service
27 , Never-married , Other-service
18 , Never-married , Other-service
18 , Never-married , Sales
30 , Never-married , Sales
43 , Never-married , Prof-specialty
32 , Never-married , Other-service
47 , Never-married , Prof-specialty
29 , Never-married , Craft-repair
30 , Never-married , Handlers-cleaners
31 , Never-married , Adm-clerical
24 , Never-married , Sales
36 , Never-married , Machine-o

 , Sales
24 , Never-married , Prof-specialty
24 , Never-married , Adm-clerical
25 , Never-married , Farming-fishing
19 , Never-married , Farming-fishing
25 , Never-married , Exec-managerial
45 , Never-married , Craft-repair
26 , Never-married , Prof-specialty
24 , Never-married , Prof-specialty
33 , Never-married , Prof-specialty
21 , Never-married , Craft-repair
45 , Never-married , Machine-op-inspct
20 , Never-married , Handlers-cleaners
35 , Never-married , Other-service
21 , Never-married , Craft-repair
29 , Never-married , Machine-op-inspct
54 , Never-married , Prof-specialty
37 , Never-married , Exec-managerial
28 , Never-married , Machine-op-inspct
23 , Never-married , Sales
31 , Never-married , Craft-repair
34 , Never-married , Other-service
29 , Never-married , Handlers-cleaners
25 , Never-married , Machine-op-inspct
26 , Never-married , Sales
46 , Never-married , Prof-specialty
30 , Never-married , Handlers-cleaners
19 , Never-married , Handlers-cleaners
26 , Never-married , 

Never-married , Transport-moving
23 , Never-married , Craft-repair
41 , Never-married , Machine-op-inspct
28 , Never-married , Prof-specialty
22 , Never-married , Handlers-cleaners
34 , Never-married , Adm-clerical
27 , Never-married , Adm-clerical
78 , Never-married , Exec-managerial
42 , Never-married , Sales
21 , Never-married , Adm-clerical
22 , Never-married , Adm-clerical
26 , Never-married , Prof-specialty
33 , Never-married , Exec-managerial
37 , Never-married , Transport-moving
29 , Never-married , Prof-specialty
42 , Never-married , Prof-specialty
23 , Never-married , Other-service
18 , Never-married , Farming-fishing
46 , Never-married , Other-service
39 , Never-married , Other-service
19 , Never-married , Craft-repair
54 , Never-married , Adm-clerical
26 , Never-married , Prof-specialty
22 , Never-married , Transport-moving
18 , Never-married , Sales
40 , Never-married , Tech-support
21 , Never-married , Other-service
31 , Never-married , Other-service
37 , Never-married , 


59 , Never-married , Craft-repair
24 , Never-married , Other-service
25 , Never-married , Prof-specialty
20 , Never-married , Transport-moving
34 , Never-married , Prof-specialty
17 , Never-married , Other-service
18 , Never-married , Transport-moving
67 , Never-married , Prof-specialty
29 , Never-married , Machine-op-inspct
37 , Never-married , Adm-clerical
24 , Never-married , Adm-clerical
21 , Never-married , Craft-repair
40 , Never-married , Craft-repair
28 , Never-married , Exec-managerial
17 , Never-married , Adm-clerical
22 , Never-married , Priv-house-serv
40 , Never-married , Adm-clerical
35 , Never-married , Craft-repair
31 , Never-married , Protective-serv
22 , Never-married , Sales
21 , Never-married , Craft-repair
32 , Never-married , Sales
24 , Never-married , Sales
42 , Never-married , Prof-specialty
47 , Never-married , Exec-managerial
19 , Never-married , Sales
33 , Never-married , Prof-specialty
33 , Never-married , Tech-support
25 , Never-married , Handlers-cleaners

Never-married , Craft-repair
29 , Never-married , Adm-clerical
21 , Never-married , Adm-clerical
28 , Never-married , Exec-managerial
34 , Never-married , Prof-specialty
28 , Never-married , Handlers-cleaners
34 , Never-married , Craft-repair
49 , Never-married , Craft-repair
24 , Never-married , Other-service
17 , Never-married , Farming-fishing
28 , Never-married , Machine-op-inspct
33 , Never-married , Craft-repair
59 , Never-married , Other-service
38 , Never-married , Adm-clerical
25 , Never-married , Other-service
40 , Never-married , Prof-specialty
22 , Never-married , Prof-specialty
40 , Never-married , Prof-specialty
35 , Never-married , Machine-op-inspct
25 , Never-married , Craft-repair
34 , Never-married , Prof-specialty
25 , Never-married , Adm-clerical
34 , Never-married , Craft-repair
38 , Never-married , Adm-clerical
29 , Never-married , Other-service
24 , Never-married , Other-service
18 , Never-married , Sales
18 , Never-married , Other-service
27 , Never-married , Ma

, Never-married , Exec-managerial
22 , Never-married , Machine-op-inspct
17 , Never-married , Sales
26 , Never-married , Protective-serv
63 , Never-married , Exec-managerial
29 , Never-married , Exec-managerial
29 , Never-married , Prof-specialty
26 , Never-married , Other-service
31 , Never-married , Exec-managerial
17 , Never-married , Other-service
37 , Never-married , Transport-moving
26 , Never-married , Prof-specialty
47 , Never-married , Prof-specialty
17 , Never-married , Sales
53 , Never-married , Prof-specialty
32 , Never-married , Craft-repair
32 , Never-married , Transport-moving
35 , Never-married , Adm-clerical
35 , Never-married , Machine-op-inspct
21 , Never-married , Machine-op-inspct
35 , Never-married , Adm-clerical
24 , Never-married , Other-service
42 , Never-married , Sales
35 , Never-married , Machine-op-inspct
36 , Never-married , Other-service
34 , Never-married , Prof-specialty
34 , Never-married , Craft-repair
29 , Never-married , Craft-repair
23 , Never-marr

Never-married , Craft-repair
19 , Never-married , Adm-clerical
17 , Never-married , Prof-specialty
24 , Never-married , Other-service
26 , Never-married , Prof-specialty
33 , Never-married , Handlers-cleaners
34 , Never-married , Adm-clerical
24 , Never-married , Transport-moving
26 , Never-married , Machine-op-inspct
37 , Never-married , Adm-clerical
27 , Never-married , Machine-op-inspct
24 , Never-married , Prof-specialty
23 , Never-married , Adm-clerical
23 , Never-married , Handlers-cleaners
63 , Never-married , Machine-op-inspct
26 , Never-married , Sales
19 , Never-married , Adm-clerical
35 , Never-married , Adm-clerical
29 , Never-married , Adm-clerical
50 , Never-married , Other-service
33 , Never-married , Sales
36 , Never-married , Craft-repair
29 , Never-married , Exec-managerial
27 , Never-married , Farming-fishing
23 , Never-married , Craft-repair
21 , Never-married , Machine-op-inspct
44 , Never-married , Prof-specialty
68 , Never-married , Adm-clerical
24 , Never-marrie

Never-married , Machine-op-inspct
71 , Never-married , Adm-clerical
26 , Never-married , Sales
18 , Never-married , Handlers-cleaners
23 , Never-married , Craft-repair
20 , Never-married , Farming-fishing
29 , Never-married , Sales
35 , Never-married , Exec-managerial
22 , Never-married , Prof-specialty
28 , Never-married , Prof-specialty
23 , Never-married , Prof-specialty
19 , Never-married , Sales
23 , Never-married , Machine-op-inspct
29 , Never-married , Prof-specialty
17 , Never-married , Handlers-cleaners
21 , Never-married , Tech-support
19 , Never-married , Handlers-cleaners
17 , Never-married , Handlers-cleaners
27 , Never-married , Other-service
53 , Never-married , Prof-specialty
24 , Never-married , Sales
86 , Never-married , Adm-clerical
25 , Never-married , Craft-repair
26 , Never-married , Craft-repair
90 , Never-married , Exec-managerial
23 , Never-married , Sales
27 , Never-married , Prof-specialty
18 , Never-married , Sales
22 , Never-married , Craft-repair
35 , Neve

 Never-married , Machine-op-inspct
27 , Never-married , Craft-repair
18 , Never-married , Sales
43 , Never-married , Exec-managerial
25 , Never-married , Adm-clerical
36 , Never-married , Other-service
22 , Never-married , Other-service
27 , Never-married , Other-service
36 , Never-married , Tech-support
76 , Never-married , Farming-fishing
40 , Never-married , Adm-clerical
22 , Never-married , Sales
17 , Never-married , Handlers-cleaners
44 , Never-married , Other-service
34 , Never-married , Craft-repair
26 , Never-married , Craft-repair
34 , Never-married , Other-service
20 , Never-married , Other-service
22 , Never-married , Handlers-cleaners
31 , Never-married , Craft-repair
19 , Never-married , Handlers-cleaners
23 , Never-married , Adm-clerical
33 , Never-married , Machine-op-inspct
29 , Never-married , Prof-specialty
19 , Never-married , Handlers-cleaners
22 , Never-married , Adm-clerical
19 , Never-married , Other-service
36 , Never-married , Transport-moving
25 , Never-marrie

, Prof-specialty
32 , Never-married , Craft-repair
51 , Never-married , Sales
18 , Never-married , Sales
29 , Never-married , Prof-specialty
31 , Never-married , Tech-support
24 , Never-married , Exec-managerial
19 , Never-married , Other-service
18 , Never-married , Machine-op-inspct
35 , Never-married , Other-service
33 , Never-married , Prof-specialty
18 , Never-married , Farming-fishing
24 , Never-married , Other-service
21 , Never-married , Adm-clerical
22 , Never-married , Craft-repair
17 , Never-married , Handlers-cleaners
30 , Never-married , Protective-serv
18 , Never-married , Adm-clerical
54 , Never-married , Handlers-cleaners
23 , Never-married , Sales
18 , Never-married , Handlers-cleaners
21 , Never-married , Adm-clerical
24 , Never-married , Craft-repair
21 , Never-married , Protective-serv
37 , Never-married , Protective-serv
18 , Never-married , Other-service
23 , Never-married , Craft-repair
22 , Never-married , Prof-specialty
33 , Never-married , Transport-moving
31 

In [19]:
# Querying Data with SQLAlchemy ORM based on filter on native_country column having a value as "Columbia"
count = 0
for rows in engine.execute("SELECT age,sex,occupation FROM adultdb where native_country = 'Columbia' ").fetchall():
    print(rows.age," ",rows.sex," ",rows.occupation)
    count+=1
print("Total Records Retrieved : ",count)

2018-07-30 18:47:18,827 INFO sqlalchemy.engine.base.Engine SELECT age,sex,occupation FROM adultdb where native_country = 'Columbia' 
2018-07-30 18:47:18,829 INFO sqlalchemy.engine.base.Engine ()
75   Female   Adm-clerical
26   Male   Prof-specialty
49   Female   Exec-managerial
38   Male   Sales
50   Male   Adm-clerical
53   Female   Machine-op-inspct
26   Female   Machine-op-inspct
43   Male   Tech-support
29   Male   Exec-managerial
64   Male   Prof-specialty
26   Male   Transport-moving
25   Female   Machine-op-inspct
23   Male   Craft-repair
49   Male   Other-service
33   Female   Machine-op-inspct
28   Male   Craft-repair
31   Male   Prof-specialty
36   Female   Adm-clerical
31   Male   Machine-op-inspct
21   Male   Adm-clerical
32   Male   Handlers-cleaners
34   Female   Machine-op-inspct
36   Male   Adm-clerical
18   Male   Handlers-cleaners
51   Male   Craft-repair
41   Female   Priv-house-serv
23   Male   Other-service
39   Female   Other-service
45   Male   Adm-clerical
45   


**Q5. Write two function queries**

In [20]:
# Querying Data with SQLAlchemy ORM - Use aggregate Function -count the number of male and female based on sex data column values
# import func module from sqlalchemy.sql package in order to perform aggregate functions
from sqlalchemy.sql import func
if __name__ == "__main__":
    session = loadSession()
    result = session.query(func.count(Adultdb.age).label('count_age'), Adultdb.sex ).group_by(Adultdb.sex).all()
    print("Count of people based on sex ")
    print('*'*80)
          
    for rows in result:
        print(rows)

2018-07-30 18:47:51,205 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-07-30 18:47:51,207 INFO sqlalchemy.engine.base.Engine SELECT count(adultdb.age) AS count_age, adultdb.sex AS adultdb_sex 
FROM adultdb GROUP BY adultdb.sex
2018-07-30 18:47:51,209 INFO sqlalchemy.engine.base.Engine ()
Count of people based on sex 
********************************************************************************
(7391, 'Female')
(16554, 'Male')


In [21]:
# Querying Data with SQLAlchemy ORM - Use aggregate Function -average age and minimum age of people based on sex column value .
if __name__ == "__main__":
    session = loadSession()
    result = session.query(func.avg(Adultdb.age).label('avg_age'),func.min(Adultdb.age).label('min_age'), Adultdb.sex).group_by(Adultdb.sex).all()
    print("Average age,minimum age  of people based on sex ")
    print('*'*80)
    for rows in result:
        print(rows)

2018-07-30 18:48:13,609 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-07-30 18:48:13,611 INFO sqlalchemy.engine.base.Engine SELECT avg(adultdb.age) AS avg_age, min(adultdb.age) AS min_age, adultdb.sex AS adultdb_sex 
FROM adultdb GROUP BY adultdb.sex
2018-07-30 18:48:13,614 INFO sqlalchemy.engine.base.Engine ()
Average age,minimum age  of people based on sex 
********************************************************************************
(37.77864970910567, 17, 'Female')
(39.699287181345895, 17, 'Male')
